# Regex in Pandas

**Inhalt:** Regular Expressions in Pandas anwenden

**Nötige Skills:** Regex in Python

**Lernziele:**
- Ein praktisches Beispiel kennenlernen, wo Regex nützlich sein kann

# Das Beispiel

Das Bundesamt für Statistik stellt oft Dateien in verknorkster Form zur Verfügung, zum Beispiel, wenn man Auswertungen nach Gemeinden, Bezirken und Kantonen über das interaktive Portal generieren lässt: https://www.pxweb.bfs.admin.ch/pxweb/de/

Das vorliegende Beispiel beinhaltet die Bevölkerungszahlen, gegliedert nach Zivilstand einerseits (ledig, verheiratet, etc) und nach räumlicher Struktur (Gemeinden, Kantone, etc) andererseits.

Das File ist gespeichert unter `dataprojects/BFS/px-x-0102010000_103.xlsx`

## Vorbereitung

In [1]:
import pandas as pd

## Datei laden

In [2]:
path = 'dataprojects/BFS/px-x-0102010000_103.xlsx'

In [3]:
df = pd.read_excel(path)

## Explorieren

In [4]:
df.head(20)

,Einheit,Bevölkerungstyp,Geschlecht,Zivilstand,Altersklasse - Total
0,Schweiz,Ständige Wohnbevölkerung,Geschlecht - Total,Ledig,3650651
1,Schweiz,Ständige Wohnbevölkerung,Geschlecht - Total,"Verheiratet, in eingetragener Partnerschaft",3583008
2,Schweiz,Ständige Wohnbevölkerung,Geschlecht - Total,"Verwitwet, durch Tod aufgelöste Partnerschaft",407408
3,Schweiz,Ständige Wohnbevölkerung,Geschlecht - Total,"Geschieden, unverheiratet, gerichtlich aufgelö...",685622
4,- Zürich,Ständige Wohnbevölkerung,Geschlecht - Total,Ledig,666873
5,- Zürich,Ständige Wohnbevölkerung,Geschlecht - Total,"Verheiratet, in eingetragener Partnerschaft",610396
6,- Zürich,Ständige Wohnbevölkerung,Geschlecht - Total,"Verwitwet, durch Tod aufgelöste Partnerschaft",63173
7,- Zürich,Ständige Wohnbevölkerung,Geschlecht - Total,"Geschieden, unverheiratet, gerichtlich aufgelö...",125889
8,>> Bezirk Affoltern,Ständige Wohnbevölkerung,Geschlecht - Total,Ledig,21785
9,>> Bezirk Affoltern,Ständige Wohnbevölkerung,Geschlecht - Total,"Verheiratet, in eingetragener Partnerschaft",23865


Offensichtlich müssen wir die Tabelle zuerst etwas umstellen, damit sie angenehm zu bearbeiten ist.

## Aufbereiten

Wir möchten:
- für jede Gemeinde, Bezirk eine Zeile
- für jeden Zivilstand eine Spalte

Die Lösung dafür kennen wir bereits: `df.pivot()`

In [ ]:
df = df.pivot(index='Einheit', columns='Zivilstand', values='Altersklasse - Total')

In [ ]:
df.head(2)

Nun verschönern wir die Sache noch etwas...

In [ ]:
df = df.reset_index()

In [ ]:
df.columns.name = None

In [ ]:
spalten = {
    'Geschieden, unverheiratet, gerichtlich aufgelöste Partnerschaft': 'Geschieden',
    'Verheiratet, in eingetragener Partnerschaft': 'Verheiratet',
    'Verwitwet, durch Tod aufgelöste Partnerschaft': 'Verwitwet'
}

In [ ]:
df = df.rename(columns=spalten)

In [ ]:
df.head(2)

## Die grographischen Einheiten...

Schauen wir uns mal näher an, was in der Spalte "Einheit" drinsteht

In [ ]:
df['Einheit']

Welche Einheitstypen gibt es? Und welches Muster haben sie?
- Gemeinde ("...... 9999 Gemeindename")
- Bezirk (">> Bezirsname")
- Kantone ("- Kantonsname")
- Land ("Land")

Ziel:
- eine Spalte "Einheitstyp"
- eine Spalte "Einheitsnummer"
- eine Spalte "Einheitsname"

## Pandas-Funktionen, die Regex brauchen können

Einige Befehle heissen leicht anders, funktionieren aber sehr ähnlich wie in der re.Library

- **`str.contains(r"regex")`**: das Pendant zu `re.search()` - ja/nein-Antwort

- **`str.extract(r"regex")`**: auch ähnlich wie `re.search()` - Suchergebnis als Antwort

- **`str.replace(r"regex", "str")`**: das Pendant zu `re.sub()` - ersetzt Match mit String

Wir wenden diese Funktionen jetzt an.

### Aber zuerst ...

Zuerst brauchen wir die Regex-Ausdrücke, um die Einheiten zu erkennen. Am besten mit Tests beginnen, ob die Regex an einer Einheit anschlägt - und jeweils auch testen, ob die Regex bei Einheiten, die wir *nicht* wollen, auch *nicht* anschlägt.

In [ ]:
# Test für Gemeinde
re.search(r"^\.{6}\d{4} .+$", "......0001 Aeugst am Albis")

In [ ]:
# Test für Bezirke
re.search(r"^>> .+$", ">> Wahlkreis Luzern-Stadt")

In [ ]:
# Test für Kantone
re.search(r"^- .+$", "- Aargau")

In [ ]:
# Test für Land
re.search(r"Schweiz", "Schweiz")

### Spalte "Einheitstyp"

Hier können wir die Funktion `str.contains()` gut brauchen.

Wir testen damit mal, ob die Einträge in der Spalte "Einheit" eine Gemeinde sind:

In [ ]:
df['Einheit'].str.contains(r"^\.{6}\d{4} .+$")

Basierend auf dieser True/False-Liste können wir nun die Tabelle filtern und mit `df.loc[]` jeweils den richtigen Eintrag in unserer neuen Spalte "Einheitstyp" erzeugen.

In [ ]:
df.loc[df['Einheit'].str.contains(r"^\.{6}\d{4} .+$"), 'Einheitstyp'] = "Gemeinde"
df.loc[df['Einheit'].str.contains(r"^>> .+$"), 'Einheitstyp'] = "Bezirk"
df.loc[df['Einheit'].str.contains(r"^- .+$"), 'Einheitstyp'] = "Kanton"
df.loc[df['Einheit'].str.contains(r"Schweiz"), 'Einheitstyp'] = "Land"

In [ ]:
df

### Spalte "Einheitsnummer"

Hier kommt die Funktion `str.extract()` gelegen.

In [ ]:
df['Einheit'].str.extract(r"^\.{6}(\d{4}) .+$")

Wir können den Output dieser Funktion für die neue Spalte setzen.

In [ ]:
df['Einheitsnummer'] = df['Einheit'].str.extract(r"^\.{6}(\d{4}) .+$")

In [ ]:
df.head(40)

**Achtung: ** Dieser Trick funktioniert hier, weil die Kantone und Bezirke *keine* Nummer haben - wir schreiben dort jetzt einfach "NaN" hin. Würden wir dieselbe Regex-Extraktion auch auf Kantone anwenden, müssten wir aufpassen, dass wir die Werte der Gemeinden, die dann ebenfalls "NaN" wären, nicht wieder überschreiben würden.

### Spalte "Einheitsname"

Es gibt zig Varianten, wie wir hier zum Ziel kommen können. Wir wählen mal den folgenden:
- Spalte "Einheit" kopieren
- Jeweils für Gemeinden, Bezirke, Kantone separat den ganzen Käse am Anfang rauslöschen, der nicht zum Namen gehört.

In [ ]:
df['Einheitsname'] = df['Einheit']

In [ ]:
df.head(2)

In [ ]:
# Gemeinden
df['Einheitsname'].str.replace(r"^\.{6}\d{4} ", "")

In [ ]:
# Gemeinden
df['Einheitsname'] = df['Einheitsname'].str.replace(r"^\.{6}\d{4} ", "")

In [ ]:
# Bezirke
df['Einheitsname'] = df['Einheitsname'].str.replace(r"^>> ", "")

In [ ]:
# Kantone
df['Einheitsname'] = df['Einheitsname'].str.replace(r"^- ", "")

In [ ]:
df